# 3 - Easy Wins 2
##### **Author: Adam Gatt**

## using

## Uniform Initialisation Syntax

## auto (in some situations)

## Const references (pre-11)
Despite introducing the useful `nullptr` in the previous notebook, the concept of the null reference has been described as its creator as the [Billion Dollar Mistake](https://www.youtube.com/watch?v=YYkOWzrO3xg).